In [1]:
import random
import hashlib
from sympy import isprime

# Parameters
p = 101  # small prime for demo
g = 2    # generator
x = 15   # secret
y = pow(g, x, p)  # public key

def hash_challenge(t, message=""):
    h = hashlib.sha256(f"{t}{message}".encode()).hexdigest()
    return int(h, 16) % 100  # reduce to small range for demo

# Honest Schnorr Protocol
def honest_schnorr():
    r = random.randint(1, p-1)
    t = pow(g, r, p)
    e = random.randint(0, 99)
    s = (r + e * x) % (p - 1)
    return t, e, s

# Verification
def verify(t, e, s):
    lhs = pow(g, s, p)
    rhs = (t * pow(y, e, p)) % p
    return lhs == rhs

# Cheating Prover
def cheating_schnorr():
    e_fake = random.randint(0, 99)
    s_fake = random.randint(1, p-1)
    t_fake = (pow(g, s_fake, p) * pow(y, -e_fake, p)) % p
    return t_fake, e_fake, s_fake

# Fiat–Shamir Non-Interactive
def fiat_shamir_schnorr(message=""):
    r = random.randint(1, p-1)
    t = pow(g, r, p)
    e = hash_challenge(t, message)
    s = (r + e * x) % (p - 1)
    return t, e, s

# Experiments
print("--- Honest Interactive Run ---")
t, e, s = honest_schnorr()
print(f"Prover commitment t = {t}")
print(f"Verifier challenge e = {e}")
print(f"Response s = {s}")
print("Verification:", "Passed" if verify(t, e, s) else "Failed")

print("\n--- Cheating Attempt ---")
t_fake, e_fake, s_fake = cheating_schnorr()
print(f"Prover fakes t = {t_fake}")
print(f"Verifier challenge e = {e_fake}")
print("Verification:", "Passed" if verify(t_fake, e_fake, s_fake) else "Failed")

print("\n--- Fiat–Shamir (Non-Interactive) ---")
t_fs, e_fs, s_fs = fiat_shamir_schnorr()
print(f"Hash-based challenge = {e_fs}")
print("Verification:", "Passed" if verify(t_fs, e_fs, s_fs) else "Failed")

print("\n--- Cheating Probability Experiment ---")
trials = 100
success = 0
for _ in range(trials):
    t_fake, e_fake, s_fake = cheating_schnorr()
    if verify(t_fake, e_fake, s_fake):
        success += 1
print(f"Cheating success rate = {success / trials} (after {trials} runs)")

--- Honest Interactive Run ---
Prover commitment t = 69
Verifier challenge e = 24
Response s = 15
Verification: Passed

--- Cheating Attempt ---
Prover fakes t = 38
Verifier challenge e = 47
Verification: Passed

--- Fiat–Shamir (Non-Interactive) ---
Hash-based challenge = 73
Verification: Passed

--- Cheating Probability Experiment ---
Cheating success rate = 1.0 (after 100 runs)
